In [1]:
%pylab inline
import sys

Populating the interactive namespace from numpy and matplotlib


In [2]:
import time

# The total chromaticity is given by :

# $$ \xi_{tot} = \frac{1}{4\pi} \oint \big[ m (s)D(s)+k(s)\big] \beta(s) ds $$

## Where m is the strength of the sextupole; D is the dispersion function; k is the quadrupole strength; $\beta$ is the beta function 

## There exists a linear relationship between the chromaticity and the sextupole strength, and the slope is proportional to the product of D and $\beta$

$  $

## $\begin{cases} \begin{align*} \xi_x &= \sum_i a_i \Delta x_i + x_0 \\
                              \xi_y &= \sum_i b_i \Delta x_i + y_0 \end{align*}\end{cases}$ 


##  Where $\xi_x$, $\xi_y$ is the first order chromaticity of the original lattice 

## $\Delta x_i$ is the variation of the strength of the sextupole

## $a_i$, $ b_i$ only depends on local optical functions

## $x_0$, $y_0$ is the first order chromaticity of the HER with Rot installed before fixing the chrom

$   $ 

In [3]:
ss = array([-3.57664, 0.8176, -3.60689, -1.7296, 0.82926, -1.69482, 0.50485067061]) # alter rot

In [4]:
a = array([0.699, 6.3548, 0.6984, 0.7042, 6.3707, 0.699, 6.3643]) #coefficient of the first equation

b = array([-1.7957, -1.9373, -1.7892, -1.7899, -1.9367, -1.7831, -1.928]) # coefficient of the second equation

CX = 1.593508     #orginal chrom.x
CY = 1.622865     #orginal chrom.y

x0 = -1.190362    #current chrom.x
y0 = -6.084533    #current chrom.y

In [5]:
def cx(x):         #compute chrom.x
    return dot(a,x) + x0     

def cy(x):         #compute chrom.y
    return dot(b,x) + y0    

$  $

# let $\Delta \xi= \xi_x - x_0 $, and  $\Delta \xi_y = \xi_y - y_0 $

# $\begin{cases} \begin{align*} \Delta \xi_x &= a_1 \Delta x_1+a_2 \Delta x_2 + ...+ a_N \Delta x_N  \\
                              \Delta \xi_y &= b_1 \Delta x_1+b_2 \Delta x_2 + ...+ b_N \Delta x_N  \end{align*}\end{cases}$ 

# N variables to be determined and exists 2 equations, so there are N-2 free variables
                              
# manipulate the second equation,  can find for any $\Delta x_j$, it can be expressed as:

# $$\begin{equation}
\Delta x_j = \frac{a_k \Delta \xi_y-b_k \Delta \xi_x +\sum\limits_{ i \not = j} \big(a_i b_k-a_k b_i\big)\Delta x_i}{a_k b_j - a_j b_k} 
\end{equation}$$
# with $j \not = k$.

# $\Delta x_1 = \frac{a_2 \Delta \xi_y-b_2 \Delta \xi_x + \big(a_3 b_2-a_2 b_3\big)\Delta x_3+...+\big(a_N b_2-a_2 b_N\big)\Delta x_N}{a_2 b_1 - a_1 b_2} $

# $\Delta x_2 = \frac{a_1 \Delta \xi_y-b_1 \Delta \xi_x +\big(a_3 b_1-a_1 b_3\big)\Delta x_3+...+\big(a_N b_1-a_1 b_N\big)\Delta x_N}{a_1 b_2 - a_2 b_1}$

In [6]:
coeff1 = array([a[i]*b[1]-a[1]*b[i] for i in arange(2,len(a))])   # coefficients of Delta x_1
coeff2 = array([a[i]*b[0]-a[0]*b[i] for i in arange(2,len(a))])

print(r'the coefficient of $\Delta x_1$ is :', coeff1)
print(r'the coefficient of $\Delta x_2$ is :', coeff2)

the coefficient of $\Delta x_1$ is : [10.01699784 10.01020986 -0.03461595  9.97707118 -0.07750399]
the coefficient of $\Delta x_2$ is : [-3.46608000e-03 -1.33918400e-02 -1.00861127e+01 -8.80740000e-03
 -1.00807015e+01]


In [7]:
def func(X0,Y0,x): # function to comptue x1 and x2; x is the array contains 4 free variables (x3,x4,x5,x6)
    Xp = CX - X0   #Chrom'.x
    Yp = CY - Y0   #Chrom'.y
    x1 = (a[1]*Yp-b[1]*Xp+dot(coeff1,x))/(a[1]*b[0]-a[0]*b[1])    # variable delta x1
    x2 = (a[0]*Yp-b[0]*Xp+dot(coeff2,x))/(a[0]*b[1]-a[1]*b[0])    # variable delta x2
    return x1, x2

# To solve the equation at global minimum, the algorithm follows:

## Step 1: Generate data set for N-2 free variables ${\Delta x_3, \Delta x_4, ..., \Delta x_N}$ within a resonable range. let $\alpha_{i} $ denote the data set for $\Delta x_i$, $\alpha_{ij} $ denotes jth element in the data set $\alpha_i$.  

## Step 2: Pick up an element $\alpha_{ij} $ for each variable $\Delta x_i$, and generate a data set for free variables $\{{\alpha_{3j},\alpha_{4j},...,\alpha_{Nj}}\}$

## Step3: Compute $\Delta x_{1 j}, \Delta x_{2 j}$ from $\{\alpha_{3j},\alpha_{4j},...,\alpha_{Nj }\}$ and only record solutions which stay in the desired range ($x_1+\Delta x_{1},x_2+\Delta x_{2} $ below the physical limit), denoted as $\Delta x_{1k}, \Delta x_{2k}$  

## Step4: Find the maximum strength in each solution set $\{ x_1 + \Delta x_{1 k},  x_2+\Delta x_{2 k}, x_3+\alpha_{3 k}, ... ,  x_N+\alpha_{N k}\}$, denoted as $X_{k}$

## Step5: make a comparison between $X_{k}$, and determine the index of the minimum in $X_{k}$ , denoted as m. Then, a local minimum solution is given by $\{x_1 + \Delta x_{1 m},  x_2+\Delta x_{2 m}, x_3+\alpha_{3 m}, ... ,  x_N+\alpha_{N m}\}$

## Iteration: reset a narrower range around $\alpha_{i m}$ to generate a new data set for $\Delta x_i$, repeat steps 2 to 5; gradually narrow the data range until a global minimum is found.

## For example, suppose it determines $\Delta x_{3}$ to be 0.5 in the first iteration. The initial data set range of $ \Delta x_3$ is $[-1.5,1.5]$. Add $\pm 0.5$ to 0.5, narrow the data range to $[0, 1]$, and  use it for the next iteration.

## Optimizer   

In [8]:
def multsolver(X0,Y0,a,b,c,d,f):

    xxi = []
    
    for i in range(num):
        for j in range(num): 
            for k in range(num):
                for l in range(num):
                    for m in range(num):
                        xx = array([a[i],b[j],c[k],d[l],f[m]]) # record free variables
                        x1 = func(X0,Y0,xx)[0]      # compute x1
                        x2 = func(X0,Y0,xx)[1]      # compute x2
                                    
                        if abs(x1) <=3 and 0<x2 <= 1 and 0<xx[2]<=1:  #condition of accpting 
                            xs = concatenate((x1,x2),axis=None) # write x1, x2 into one array
                            xxi.append(concatenate((xs,xx),axis=None))#generate the solution array 
                        else:
                            pass
                        
    xxi = array(xxi)
    
    xxim = []
    for i in range(len(xxi)):
        xxim.append(max(abs(xxi[i]+ss))) # find the maxium value of N variables in each solution
    xxim = array(xxim)
    
    if len(xxim)==0:
        sys.exit('The optimizer can not find a solution in this given range, please increase the maximum strength')
        # if no solution, increase the max 
    nm = where(xxim == min(xxim))[0][0]      # find where the minimum solution is
    return xxi[nm]         # return solution

In [9]:
def opt(x0,y0,mm,ni,num):   # sextupole strength optimizer
    
    ama =  mm   ## max of the data 
    ami = -mm   ## min of the data 
    bma =  mm
    bmi =  mm
    cma =  mm
    cmi = -mm
    dma =  mm
    dmi = -mm
    fma =  mm
    fmi = -mm

    
    sa = []  # initialize solution array
    msa = [] # initialize maximum strength array
    
    for i in range(ni):
        a = linspace(ami,ama,num)     # generating data within the required range 
        b = linspace(bmi,bma,num)
        c = linspace(cmi,cma,num)
        d = linspace(dmi,dma,num)
        f = linspace(fmi,fma,num)
        
        s = multsolver(x0,y0,a,b,c,d,f) # compute x1,x2
    
        ms = max(abs(s+ss))            # find the max in solution array
    
        ra = 1- i/10                    # narrowing the variation range of data array
    
        if ra<0.1:                     # set minimum variation range
            ra=0.1

        if i!= 0 and ms >= msa[-1]:    # if iteration step finds a bigger strength, 
                                       #go back to previous solution and increase num
            num += 1
        
            ama = sa[-1][2]+ra
            ami = sa[-1][2]-ra
        
            bma = sa[-1][3]+ra
            bmi = sa[-1][3]-ra
        
            cma = sa[-1][4]+ra
            cmi = sa[-1][4]-ra
        
            dma = sa[-1][5]+ra
            dmi = sa[-1][5]-ra
            
            fma = sa[-1][6]+ra
            fmi = sa[-1][6]-ra
            
        else:
            ama = s[2]+ra
            ami = s[2]-ra
        
            bma = s[3]+ra
            bmi = s[3]-ra
        
            cma = s[4]+ra
            cmi = s[4]-ra
        
            dma = s[5]+ra
            dmi = s[5]-ra
            
            fma = s[6]+ra
            fmi = s[6]-ra
            
        
        if num >= 15:    #insurance of efficiency
            num = 15
            
        sa.append(s)
        msa.append(ms)

        print('iteration step: %s'%(i+1))
        print('the max strength is:',ms)
        print(s+ss)
        print(num)
        print('')
        print('')
    
    sa = array(sa)
    msa = array(msa)

    nm = where(msa ==min(msa))[0][0]      # find the where the minimum solution is

    xs = sa[nm]
    print('the iretation step reached mininum:',nm+1)
    print('the max strength in the solution is:',min(msa))
    return xs

In [10]:
mm = 3   # max variation of sextupole strength
ni = 15    # number of iterations to do
num  = 8   # number of data to be generated

In [11]:
t0=time.time()

In [12]:
solution = opt(x0,y0,mm,ni,num)

iteration step: 1
the max strength is: 6.60689
[-6.00004578  0.99063039 -6.60689     1.2704      1.25783143 -4.69482
  0.9334221 ]
8


iteration step: 2
the max strength is: 5.60689
[-5.29591627  1.56508691 -5.60689     0.2704      1.11497429 -5.40910571
  0.50485067]
8


iteration step: 3
the max strength is: 5.221175714285714
[-5.16638593  1.56588379 -5.22117571 -0.6296      0.98640286 -5.02339143
  0.6334221 ]
8


iteration step: 4
the max strength is: 4.909105714285715
[-4.82625765  1.79589917 -4.87831857 -1.4296      0.87211714 -4.90910571
  0.51913638]
8


iteration step: 5
the max strength is: 4.720199628136784
[-4.72019963  0.99453728 -4.57831857 -2.1296      0.97211714 -4.60910571
  1.21913638]
8


iteration step: 6
the max strength is: 4.492604285714286
[-4.46838613  1.68244825 -4.49260429 -2.7296      0.88640286 -4.35196286
  0.61913638]
8


iteration step: 7
the max strength is: 4.280534285714286
[-4.25531425  1.68293921 -4.27831857 -3.2296      0.95783143 -4.28053429
  0.5

In [13]:
t1=time.time()
print(t1-t0)

10.281648874282837


In [14]:
ss #original sextupoles

array([-3.57664   ,  0.8176    , -3.60689   , -1.7296    ,  0.82926   ,
       -1.69482   ,  0.50485067])

In [15]:
solution  #variation of the sextupoles strength

array([-0.4309688 ,  0.23606408, -0.4       , -2.28571429,  0.02857143,
       -2.31428571,  0.77142857])

In [16]:
solution+ss #solution array 

array([-4.0076088 ,  1.05366408, -4.00689   , -4.01531429,  0.85783143,
       -4.00910571,  1.27627924])

In [17]:
solution = array([-0.41964882,  0.58336436, -0.34550048, -2.34350763,  0.59863976,
       -2.32757282, -0.1453647 ])

In [18]:
solution+ss

array([-3.99628882,  1.40096436, -3.95239048, -4.07310763,  1.42789976,
       -4.02239282,  0.35948597])

# Orginal chromaticity  $\xi_x = 1.593508$   

In [19]:
cx(solution) # testing the solution

1.5935080591119997

# Orginal chromaticity $ \xi_y = 1.622865$

In [20]:
cy(solution)

1.6228649909489992

# If the chrom matches above, use this solution as a initial guess to runing an optimization in Tao.